# ShrimpSimulator
This simulator simulates the Shrimp model.

## Relevant Equations 

### Newton-Euler (equations 2.2 and 2.3)
$$\mathbf{\dot{v}^I_F} = (\mathbf{f_{aero}} + m \mathbf{g} - m \mathbf{\omega^{FI} }\times \mathbf{v^I_F})$$

$$\mathbf{\dot{\omega}^{FI}} = \mathbf{I}^{-1}(\mathbf{\tau_{aero}} - \mathbf{\omega^{FI} }\times \mathbf{I\omega^{BI}})$$

* $v_F^I$ is velocity of the center of mass of flyer frame in inertial/world frame
* $\mathbf{\omega^{FI} } = [p\ q\ r]^T$ is the angular velocity of flyer frame w.r.t. inertial
* $\mathbf{\omega^{BI} }$ is the angular velocity of body frame wrt inertial (same as flyer but with rotation about z axis).


### Blade Equations (equations 3.8)
$$[v_E^I]^E = \mathbf{R^E_F}[v^I_F]^F + \mathbf{R^E_F}[\omega^{SF}]^F \times \mathbf{R^E_F} [S_{EF}]^F + \mathbf{R^E_F}[\nu]^F$$
$$\gamma = arctan(\frac{[v_E^I]^E [\hat{z}]^E}{[v_E^I]^P [\hat{x}]^E})$$


* $[\nu]^F = [0\ 0\ \nu]^T$ is induced inflow velocity
* $\nu$ is the induced inflow velocity (Equation 3.1, thesis p.52)
* $[v_E^I]^E$ is the relative wing in the propeller frame.


### Euler Angles Convention
For euler angles, we are using the `scipy` library of `spatial.transform` with `Rotation`. https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html#scipy.spatial.transform.Rotation

This uses the capital notation `ZXY` to denote intrinsic rotations, which we will use such that the final rotation is about the rotational axis of the vehicle for readibility. This is such that the first rotation is about the Z axis, or corresponds to the yaw.
This matches the MEAM 620 project 1 convention.
Additionally, the euler angles are $\phi, \theta, \psi$, where $\phi$ is yaw.

Please use the `euler2Rotm` and `rotm2Euler` functions for converting between rotation matricies and euler angle vectors 

## Code parameters

* camelCase unless stated otherwise
* Underscores for frames or for subscripts


## Notes
Flyer frame means that it is *not* in the frame of the propellor with the extra spin on the propellor. This is 
only relevant for the shaft propellor.

For the body drag plates, this is all in the body frame (no extra spin) so the euler angular rates are still correct, as well as the euler angles. 

Notation 
* For angular velocities: $\omega$ is a 1x3 vector, while $[\omega]$ is the tensor. We will try to have $[]$ always denote a tensor. 
* For Rotation Matricies: ${}^A R_B$ denotes a rotation from $B$ to $A$, e.g. $p_A = {}^AR_B p_B$
* For position/velocity vectors: ${}^c v_a^b$ denotes a vector from $a$ to $b$ in the $c$ frame.


TODO: read through the math extra carefully again. 
Be explicit with the notation and work through each sub example - write an example for
that one and the next one.

In [5]:
%matplotlib inline
import numpy as np 
from scipy.spatial.transform import Rotation as R
from enum import Enum

In [6]:
def rotm2Euler(x):
    """Rotation matrix to Euler Angles ZXY intrinsic vector for Shrimp"""
    r = R.from_dcm(x)
    return r.as_euler('ZXY')
    
def euler2Rotm(x):
    """Euler angles ZXY intrinsic vector to Rotation Matrix for Shrimp"""
    r = R.from_euler('ZXY', x)
    return r.as_dcm()

def quat2Euler(x):
    """Quaterion to Euler Angles ZXY intrinsic vector for Shrimp"""
    r = R.from_quat(x)
    return r.as_euler('ZXY')

def euler2Quat(x):
    """Euler angles ZXY intrinsic vector to Quaternion for Shrimp"""
    r = R.from_quat('ZXY', x)
    return r.as_euler()

def getYaw(eulerAngle):
    return eulerAngle[0]

def getPitch(eulerAngle):
    return eulerAngle[1]

def getRoll(eulerAngle):
    return eulerAngle[2]

def rotateTensor(tensor, rotationMatrix):
    """ Rotate those tensors!
        Tensors (like inertia tensors) rotate as I_B = R_A^B I_B (R_A^B)^T
        Where 
        I_A is the tensor in frame A
        I_B is the tensor in frame B
        R_A^B is the rotation matrix from frame A to frame B
        
        Note this doesn't do the parallel axis theorem, so still be careful!
    """
    invRot = np.transpose(rotationMatrix)
    firstMultiplication = rotationMatrix.dot(tensor)
    return firstMultiplication.dot(invRotation)

eul = np.array([np.pi/2, 0, 0])
k = [0,2,0]
r = euler2Rotm(eul)
print(r.dot(k))

rot = R.from_euler('z', 45, degrees='true')
rotm = R.as_dcm(rot)
print(r.dot(k))
print(rotm)



[-2.0000000e+00  4.4408921e-16  0.0000000e+00]
[-2.0000000e+00  4.4408921e-16  0.0000000e+00]
[[ 0.70710678 -0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]]


## Relative Wind
Calculating the relative wind is pretty similar to the blade equations above. 
We want to take the given blade section, and then calculate the relative wind vector in the propellor frame. 

![relativewind](../lib/relative_wind.png)

The propellor (E) frame described:

![propellorFrame](../lib/element_frame.png)

Assumptions: No exterior wind disturbance.

The relative wind in this case comes from three components, which we can sum independently:
* **The body velocity**. This is simply the velocity of the body frame w.r.t. the world frame in the world frame $^w v_w^b$ rotated to the element frame $^e v_w^b$. 

$${}^e v_w^b = {}^e R_b {}^b R_w {}^w v_w^b$$

* **The inflow velocity**. The inflow velocity $\nu$ adds simply another component in the negative z-axis, coming from the ducted interaction:

$${}^e v_{inflow} = [0,\ 0,\ -\nu]^T$$

* **The body angular velocity**. The angular velocity in the body frame $\hat{\omega}^b$ needs to be translated to a linear velocity at the element point. The position vector from the body origin (center of mass) to the element origin in the body frame is ${}^b p_b^e$.
${}^b p_b^e = [r \cos\kappa,\ r \sin\kappa,\ h]^T$, where $h$ is the height (z-axis distance) from the cg to the propellor center, $r$ is the radius of the element, and $\kappa$ is the angle of the propellor with respect to the body x axis. 

For calculating the angle of the propellor $\kappa$, we have to take a different approach for body vs. shaft blades.

For the body blades, the first blade is always aligned with the body x-axis and all further blades are spaced evenly around the center of the body. 

The linear velocity of the element relative to the body in the body frame is ${}^b v_e^b = {}^b \omega^b \times {}^b p_b^e$. 

However, if the blades are rotating independently of the body rotation, with some speed $\dot{\psi}$, then there is an extra rotation of the element in the body frame: ${}^b \omega_e = [0,\ 0,\ \dot{\psi}]^T$, such that:

$${}^w v_e^b = {}^w R_b \left(({}^b \omega_b + {}^b \omega_e) \times {}^b p_b^e\right)$$

For the body blade $\dot{\psi}$ is zero, while otherwise it is the propellor spin rate in radians per second.

A note on $\kappa$ for the shaft blade:
For the shaft blades, they are spinning at roughly 3-8 revolutions per second. That is just barely fast enough that we should not track the position of each blade to do the calculation, but instead do the integral. TODO. 

The total relative wind is then:

$${}^e v_{wind} = - {}^e v_w^b - {}^e v_b^e + {}^e v_{inflow}$$

### Calculating rotation matricies
Assuming we know our XYZ intrinsic euler angles, ${}^b R_w$ is simply the transformation from euler to rotation. The rotation from body to element, ${}^e R_b$ is the rotation by the propellor angle. In the picture above, the propellor angle is 0 (the propellor y-axis aligns with the body x-axis), so ${}^e R_b$ is a rotation about the body $z$ axis by $\pi/2$. 

### Calculating angular velocity
The angular velocities we are keeping track of are those of the body frame in the world frame: ${}^w \omega_b = [p\ q\ r]^T$. To get it in the body frame, I am pretty sure you can just rotate it. But to go from the euler angle rotations:

$$\begin{bmatrix} p \\ q \\ r \end{bmatrix} = \begin{bmatrix} \cos \theta & 0 & \cos\phi \sin\theta \\ 0 & 1 & \sin\phi \\ \sin\theta & 0 & \cos\phi \cos\theta \end{bmatrix} 
\begin{bmatrix} \dot{\phi} \\ \dot{\theta} \\ \dot{\psi}\end{bmatrix}$$

In [7]:
def getRelativeWind(bladeIndex, numBlades, bladeRadius, vel_b2w_w, 
                    rot_w2b, angvel_b2w_b, inflowVelocity, height_b2e,
                    prop_angvel):
    """
    Returns relative wind angle as well as the wind angle vector for 
    the given quantities
    
    Return values:
    relativeWind_e : the relative wind in the propellor frame
    relWindAngleGamma: the wind angle relative to the propellor blade disk angle
    r_b2e_b: the vector from the body to the element in the body frame
    rot_b2e: the rotation matrix from the body to element frame 
    
    Arguements:
    bladeIndex: the index of the blade to be used. Assumes blade 0 is aligned with the 
    x-axis of the body
    numBlades: total number of blades, assuming equal spacing
    bladeRadius: the radius to do this overall calculation
    vel_b2w_w: velocity of the body relative to the world in the world frame
    rot_b2w: DCM of body to world 
    angvel_b2w_w: angular velocity of body relative to world in the world frame (p,q,r)
    inflowVelocity: also called nu - scalar amount of inflow velocity
    height_b2e: the distance from body to element along the body z axis 
    prop_angvel: propellor angular velocity (spin rate)
    
    Notation:
    _e is prop element frame
    _b is the body frame (about the center of gravity)
    _w is world/inertial frame
    
    """
    # Calculate Rotation Matricies
    propellor_angle = bladeIndex/numBlades*2*np.pi
    rot_b2e = R.from_euler('Z', propellor_angle + np.pi/2).as_dcm()
    
    # Body Velocity Calculation
    vel_b2w_b = rot_w2b.dot(vel_b2w_w)
    vel_b2w_e = rot_b2e.dot(vel_b2w_b)
    
        
    # Inflow velocity Calculation
    # it is always in the negative z direction
    vel_inducedInflow_e = np.array([0, 0, -inflowVelocity])
    
    
    # Velocity from rotational component Calculation
    # Calculate the vector from the cg to the element 
    pos_x_blade = bladeRadius*np.cos(propellor_angle + np.pi/2)
    pos_y_blade = bladeRadius*np.sin(propellor_angle + np.pi/2)
    r_b2e_b = np.array([pos_x_blade, pos_y_blade, height_b2e])
    
    # Calculate the angular velocity in the body frame
    angvel_spin_b = np.array([0, 0, prop_angvel])
    # Total angular velocity of the propellor is body + prop
    total_angvel_b = angvel_spin_b + angvel_b2w_b
    
    vel_e2b_b = np.cross(total_angvel_b, r_b2e_b)
    vel_e2b_e = rot_b2e.dot(vel_e2b_b)
    
    # Sum all forces together
    relativeWind_e = -(vel_b2w_e + vel_e2b_e) + vel_inducedInflow_e
    # TODO: confirm this is the arctan we want I guess
    relWindAngleGamma = np.arctan2(relativeWind_e[2], relativeWind_e[0])

    return (relativeWind_e, relWindAngleGamma, r_b2e_b, rot_b2e)

In [8]:
def testRelativeWind():
    
    # Test for relative wind 
    bodyEulerAngles_w = np.pi/180*np.array([0,0,0]) # ZXY
    # Below is technically body2world, but no big deal
    rot_body2world = euler2Rotm(bodyEulerAngles_w) 

    numBlades = 2
    bladeRadius = 0.1
    bladeIndex = 1
    vel_b2w_w = np.array([0, 10, 10])
    angvel_b2w_b = np.array([0.0, 0.0, 0.0])
    inflowVelocity = 0
    height_b2e = 2
    prop_angvel = 0

    # Using XYZ euler angles such that the last angle Z is the correct psi angle
    (relWind, gamma, _, _) = getRelativeWind(bladeIndex, numBlades, bladeRadius, 
                                             vel_b2w_w, rot_body2world, angvel_b2w_b, 
                                             inflowVelocity, height_b2e, prop_angvel)

    print('gamma', gamma*180/np.pi)
    print('relwind', relWind)

testRelativeWind()

gamma -135.0
relwind [-1.00000000e+01  2.22044605e-15 -1.00000000e+01]


## Lift and Drag
Note that lift is POSITIVE in this model. Z is up. 
$$L = \frac{1}{2} \rho V^2 C_L S$$
$$D = \frac{1}{2} \rho V^2 C_D S$$
Where 
* $C_L, C_D$ are lift and drag coefficients, functions of angle of attack ($alpha$)
* $S$ is wing area. 
* $\rho$ is air density 
* $V$ is velocity at the wing element

In [9]:
def getAngleOfAttack(bladePitch, relWindAngleGamma):
    """ Returns angle of attack between -pi and pi"""
    unwrapped_angle = bladePitch + relWindAngleGamma
    return (unwrapped_angle + np.pi) % (2 * np.pi ) - np.pi

def clFlatPlate(alpha):
    """ Coeff of lift for a Flat Plate """
    return 2.*np.sin(alpha)*np.cos(alpha)


def cdFlatPlate(alpha):
    """ Coeff of drag for a Flat Plate """
    return 2.*np.sin(alpha)**2.0


def cmFlatPlate(alpha):
    """ Coeff of pitch moment for a Flat Plate """
    return 0.0

def getLiftDragElement(elementWidth, chord, rho, relWind_e, alpha):
    """ Returns magnitude of lift and drag
    Arguements:
    elementWidth: the width of the blade section 
    chord: airfoil chord
    rho: air density
    relWind_e : relative wind in the propellor frame
    alpha: angle of attack of section
    Returns:
    (lift, drag) scalars
    """
    # velocity only incorporates the x and z components in the wind frame 
    speed = relWind_e.dot([1,0,1])    
    coeffLift = clFlatPlate(alpha)
    coeffDrag = cdFlatPlate(alpha)
    quantity = 0.5*speed*speed*chord*elementWidth*rho
    lift = quantity * coeffLift
    drag = quantity * coeffDrag
    return (lift, drag)


In [10]:
def rpm2RadiansPerSecond(x:float) -> float:
     return 0.10472 * x

In [20]:
class PropellorType(Enum):
    SHAFT = 1
    BODY = 2

class BladeElementParameters:
    def __init__(self, rho, pitch, width, radius, chord, index, numBlades, 
                 height_body2element, inflowVel, spinVel):
        self.rho = rho
        self.pitch = pitch 
        self.width = width  # this is in the radial direction (strip model width)
        self.radius = radius  # this should probably "middle" of the overall radius width 
        self.chord = chord
        self.index = index # Blade Index out of n blades
        self.numBlades = numBlades # total number of blades. A normal prop has 2
        self.height_body2element = height_body2element # scalar [m]
        self.inflowVel = inflowVel # Inflow velocity for this element. Scalar [m/s]
        self.spinVel = spinVel # spin velocity for this element. Scalar [m/s] 

class PropellorParameters:
    """Parameters of all blades in the propellor
       Assumes propellor center is aligned in the z-axis with the body center of mass.
    """
    def __init__(self, numBlades, pitchRootTip, radiusRootTip, chordRootTip, 
                 height_b2p, propType):
        self.numBlades = numBlades 
        self.pitchRootTip = pitchRootTip # Tuple of blade pitch, (root, tip)
        self.radiusRootTip = radiusRootTip # Tuple, (root, tip)
        self.chordRootTip = chordRootTip # Tuple, (root, tip)
        self.height_b2p = height_b2p # Z-axis distance from cg to propellor, +z is up
        self.propType = propType # PropellorType 

class BatteryParameters:
    """Parameters for a battery"""
    def __init__(self, voltage, resistance):
        self.voltage = voltage  # nominal voltage
        self.resistance = resistance  # internal resistance, assumed constant heh
       
class MotorParameters:
    """Parameters for a motor"""
    def __init__(self, K_t, resistance):
        self.K_t = K_t  # Eletromotive forceconstant, or 1/K_v
        self.resistance = resistance  # stator resistance, assumed constant heh
        
class ShrimpParameters:
    """ Parameters of Shrimp Aircraft.
    """
    def __init__(self, bodyBladesParams, shaftBladesParams, mass, Ixx, Izz, propIxx, propIzz,
                 rho, motorParams, batteryParams):
        self.bodyBladesParams = bodyBladesParams
        self.shaftBladesParams = shaftBladesParams
        self.mass = mass
        self.inertia_b = np.diag([Ixx, Ixx, Izz])  # body Inertia without prop inertia
        # Assumes prop inertia roughly rotationally symmetric for all intents and purposes
        self.propInertia_b = np.diag([propIxx, propIxx, propIzz])  
        self.rho = rho # Air density
        self.motorParams = motorParams 
        self.batteryParams = batteryParams
        
class ShrimpState:
    """ Shrimp state object. Stores state in a clear format, and also converts 
        the clear format to a numpy vector for ODE integration.
        
    """
    def __init__(self, r_b2w_w, vel_b2w_w, euler_b2w_w, angvel_b2w_w, inflowVel, prop_angVel):
        self.r_b2w_w = r_b2w_w  # np.array 3x1 position vector from world origin to body/cg
        self.vel_b2w_w = vel_b2w_w  # np.array 3x1 velocity vector from world origin to body/cg
        self.euler_b2w_w = euler_b2w_w  # np.array 3x1 euler from world to body frame
        self.angvel_b2w_w = angvel_b2w_w  # np.array 3x1 euler velocity vector from world to body frame
        self.inflowVel = inflowVel # scalar inflow velocity adjustment due to duct on the shaft propellor [m/s]
        self.prop_angVel = prop_angVel # the speed of the spin of the propellor. Scalar [rad/s]
        
        # Define some other nice things
        yawAngle = getYaw(euler_b2w_w)
        self.rot_b2f = R.from_euler('z', yawAngle).as_dcm()
        self.rot_b2w = euler2Rotm(euler_b2w_w)
        print(self.rot_b2f)
        print(self.rot_b2w)
        self.rot_f2w = np.transpose(self.rot_b2f).dot(self.rot_b2w)
    
    def getRotsFromEuler(euler_b2w_w):
        yawAngle = getYaw(euler_b2w_w)
        rot_b2f = R.from_euler('z', yawAngle).as_dcm()
        rot_b2w = euler2Rotm(euler_b2w_w)
        rot_f2w = np.transpose(self.rot_b2f).dot(self.rot_b2w)
        return (rot_b2f, rot_b2w, rot_f2w)
    
    def get_rotb2w():
    
    def fromOdeState(odeState):
        """ Converts from vector to this object.
            odeState[0] = x body position in world frame
            odeState[1] = y body position in world frame
            odeState[2] = z body position in world frame
            odeState[3] = x body velocity in world frame
            odeState[4] = y body velocity  in world frame
            odeState[5] = z body velocity  in world frame
            odeState[6] = Z ZYX euler angle in world frame
            odeState[7] = Y ZYX euler angle in world frame
            odeState[8] = X ZYX euler angle in world frame
            odeState[9] = p is x roll rate in flyer frame
            odeState[10] = q is y pitch rate in flyer frame
            odeState[11] = r is z yaw rate in flyer frame
            odeState[12] = bodyYawRate is body yaw rate wrt flyer frame
            odeState[13] = propAngRate is the prop speed wrt body frame (stator frame)
        """
        r_b2w_w = odeState[0:3]
        vel_b2w_w = odeState[3:6]
        euler_b2w_w = odeState[6:9]
        angvel_b2w_f = odeState[9:12] + np.array([0, 0, odeState[12]])
        rot_b2w = 
        angvel_b2w_w = rot_f2w.dot(angvel_b2w_f)
        inflowVel = 0
        prop_angVel = odeState[13]
        return ShrimpState(r_b2w_w, vel_b2w_w, euler_b2w_w, angvel_b2w_w, inflowVel, prop_angVel)
    

def dummyShrimpState():
    r_b2w_w = np.array([0.1, 0.1, 0.1])
    vel_b2w_w = np.array([0.1, 0, 0])
    euler_b2w_w = np.array([10, 0, 40]) * np.pi/180.
    angvel_b2w_w = np.array([0.01, 0, 0.02])
    inflowVel = 1
    prop_angVel = rpm2RadiansPerSecond(500)
    state = ShrimpState(r_b2w_w, vel_b2w_w, euler_b2w_w, angvel_b2w_w, inflowVel, prop_angVel)
    return state

temp = dummyShrimpState()
temp2 = np.zeros(13)
temp3 = ShrimpState.fromOdeState(temp2)

[[ 0.98480775 -0.17364818  0.        ]
 [ 0.17364818  0.98480775  0.        ]
 [ 0.          0.          1.        ]]
[[ 7.54406507e-01 -1.73648178e-01  6.33022222e-01]
 [ 1.33022222e-01  9.84807753e-01  1.11618897e-01]
 [-6.42787610e-01 -6.93889390e-18  7.66044443e-01]]


NameError: name 'self' is not defined

In [12]:
def getElementForceMoment(params, state):
    """Returns force and moment vectors in the flyer frame 
    Assumes the element radius is the radius at the center of the width. 
    The width is only used in a surface area calculation. 
    Arguments:
        params (BladeElementParameters)
        state (ShrimpState)
    """
    
    rho = params.rho
    bladeIndex = params.index
    bladeRadius = params.radius
    vel_b2w_w = state.vel_b2w_w
    angvel_b2w_w = state.angvel_b2w_w
    inflowVelocity = params.inflowVel
    height_b2e = params.height_body2element
    prop_angvel = params.spinVel
    euler_b2w = state.euler_b2w_w
    numBlades = params.numBlades
    
    rot_b2w = state.rot_b2w
    rot_w2b = np.transpose(rot_b2w)
    
    angvel_b2w_b = rot_w2b.dot(angvel_b2w_w) # TODO: check if this is the proper way to rotation ang vel vector
    
    # TODO: make call less ugly. Maybe move r_b2e_b and rot_b2e into the creation of the blade element object
    (relativeWind_e, gamma, r_b2e_b, rot_b2e) = getRelativeWind(bladeIndex, numBlades, 
                                                                bladeRadius, vel_b2w_w, 
                                                                rot_w2b, angvel_b2w_b, 
                                                                inflowVelocity, height_b2e,
                                                                prop_angvel)
    
    alpha = getAngleOfAttack(gamma, params.pitch)
    (lift, drag) = getLiftDragElement(params.width, params.chord, rho, relativeWind_e, 
                                      alpha)
    
    # Forces in the propellor frame
    normalForce_e = lift*np.cos(gamma) + drag*np.sin(gamma)
    tangentialForce_e = -lift*np.sin(gamma) + drag*np.cos(gamma)
    
    # Z-axis is up! 
    forces_e = np.array([-tangentialForce_e, 0, normalForce_e])
    
    rot_e2b = np.transpose(rot_b2e)
    forces_b = rot_e2b.dot(forces_e)
    moments_b = np.cross(forces_b, r_b2e_b)
    
    return(forces_b, moments_b)
    
def testBladeElement(radius, chord, pitch, width):
    rho = 1.225
    bladeIndex = 1
    numBlades = 2
    height_body2element = 0.01
    
    inflowVel = 1
    prop_angVel = rpm2RadiansPerSecond(500)
    # Set up this particular propellor element
    bladeParams = BladeElementParameters(rho, pitch, width, radius, chord, bladeIndex,
                                         numBlades, height_body2element, inflowVel, prop_angVel)
    
    state = dummyShrimpState()
    
    (forces_b, moments_b) = getElementForceMoment(bladeParams, state) 
    return (forces_b, moments_b)
    
def testBlade():
    n = 5 # The number of sections to split the blade into for element theory
    radiusRoot = 0
    radiusTip = 0.1 # maximum blade radius / tip to root
    width = (radiusTip-radiusRoot)/n
    chordRoot = 0.1 # Chord at the root of the blade
    chordTip = 0.2 # Chord at the tip of the blade
    pitchRoot = 0*np.pi/180
    pitchTip = 15*np.pi/180
    
    radii = np.linspace(radiusRoot, radiusTip, n)
    chords = np.linspace(chordRoot, chordTip, n)
    pitches = np.linspace(pitchRoot, pitchTip, n)
    
    forceMomentTuples = [testBladeElement(r, c, p, width) for (r,c,p) in zip(radii, chords, pitches)]

    forces = sum([f for f,_ in forceMomentTuples])
    moments = sum([m for _,m in forceMomentTuples])
    print('Forces', forces)
    print('Moments', moments)
testBlade()

Forces [-6.59186118e-19  2.96871035e-03 -1.94438323e-02]
Moments [-1.08690834e-03  2.11707116e-19  9.41867003e-20]


# ODE integration
For the ODE, we need the equations of motion of the system. We're going to do this in the body frame so we have a constant inertial tensor, and never have to "pay the price". In particular Piccoli does this as well as he does the ODE integration in the flyer frame, which I think ignores some math in terms of how angular velocities rotate as they are integrated (see this stackexchange: https://math.stackexchange.com/questions/773902/integrating-body-angular-velocity).

I think the right thing to do is integrate in the world frame. This makes the ODE more stable, at the penalty of a time-varying inertial tensor. 

Reproducing the Newton Euler Equations here:

# Newton-Euler (equations 2.2 and 2.3)
$$\mathbf{\dot{v}^I_F} = (\mathbf{f_{aero}} + m \mathbf{g} - m \mathbf{\omega^{FI} }\times \mathbf{v^I_F})$$

$$\mathbf{\dot{\omega}^{FI}} = \mathbf{I}^{-1}(\mathbf{\tau_{aero}} - \mathbf{\omega^{FI} }\times \mathbf{I\omega^{BI}})$$

* $v_F^I$ is velocity of the center of mass of flyer frame in inertial/world frame
* $\mathbf{\omega^{FI} } = [p\ q\ r]^T$ is the angular velocity of flyer frame w.r.t. inertial
* $\mathbf{\omega^{BI} }$ is the angular velocity of body frame wrt inertial (same as flyer but with rotation about z axis).



## On Flyer vs Body frame:
In Piccoli's code, the transformation between body and flyer frame is `Rb_f`, defined in `ComputeAero.m`. It is a rotation around the z axis by (yaw + prop angle). He does this weird thing where he post-multiplies a body force by `Rb_f` to get the force in the flyer frame. This is a transformation of a row matrix form the flyer frame into the body frame, so `Rb_f` properly is a rotation from the flyer frame into the body frame. 

* `b*Rb_f` yields a column in the flyer frame, b is a row vector
* `Rb_f*f` yields a column in the body frame, f is a column vector
* `f*Rb_f'` yields a row vector in the body frame, f is a row vector

## Newton's equations:
There is one concern of doing things in the body frame that timestep/numerical issue will come up if the integration is too slow, but we'll have to check that later. TODO. The Flyer frame is ill-defined when it says it neglects yaw.

We'll add up all the forces in the body frame however, and then integrate them in the world frame.

The following is all in the Body frame:
$$\mathbf{{}^b\dot{v}_w^b} = (\mathbf{{}^b f_{aero}} + m {}^b \mathbf{g} - m \mathbf{{}^b\omega^{w} }\times \mathbf{{}^b v_b^w})/m$$

### Other notes from Piccoli's simulator
Piccoli basically takes the inflow velocity $\nu$ tobe constant, as $\dot{\nu}$ almost always equals 0.

### Motor calculations
Piccoli calculates the motor moments using the following equations:
$$i_m = \frac{v_{clamp} pwm -K_{t_m}*(\omega_r)}{pwm^2 R_b+R_m}$$
where
* $i_m$ is motor current
* $R_b$ is battery resistance
* $R_m$ is motor resistance
* $K_{t_m}$ is eletromotive force constant, or $1/K_v$ 
* $pwm$ is motor pwm
* $\omega_r$ is the rotor speed wrt stator, or rather the "prop speed"


* $^F M_{motor} = R_P^F \begin{bmatrix}0 \\ 0 \\ K_{t_m} i_m \end{bmatrix} $
In this case, the moment only exists in the z axis anyway, and prop frame to flyer should be a z-axis rotation, so we can skip it for sanity. 

In generla the $R_P^F$ should be benign for aerodynamic forces.


### Integrating Angular velocity
Apparently this isn't as easy as hitting integrate:

https://www.ashwinnarayan.com/post/how-to-integrate-quaternions/
https://math.stackexchange.com/questions/773902/integrating-body-angular-velocity
 https://physics.stackexchange.com/questions/351435/newton-euler-equations-in-frame-not-centered-at-center-of-mass

In [13]:
def getBladeForceMoment(propParams, shrimpParams, state, bladeIndex):
    """Get the force moment of a single blade.
       Splits up the blade into n sections and evaluates a strip model for each
       section. Also decides inflow velocity and angular velocity based off of whether 
       it is the body or shaft propellor
       Arguments:
           propParams: PropellorParameters
           shrimpParams: ShrimpParameters
           state: ShrimpState
           bladeIndex: index of the blade in the propellor
       
    """
    
    n = 5 # The number of sections to split the blade into for element theory
    
    (radiusRoot, radiusTip) = propParams.radiusRootTip
    width = (radiusTip-radiusRoot)/n
    
    pitches = np.linspace(propParams.pitchRootTip[0], propParams.pitchRootTip[1], n)
    radii = np.linspace(radiusRoot, radiusTip, n)
    chords = np.linspace(propParams.chordRootTip[0], propParams.chordRootTip[1], n)
   
    # Assign propellor specific characteristics
    if propParams.propType == PropellorType.SHAFT:
        inflowVel = state.inflowVel
        prop_angVel = state.prop_angVel
    elif propParams.propType == PropellorType.BODY:
        inflowVel = 0
        prop_angVel = 0
    else:
        raise Exception('Using unimplemented PropellorType!')
    
    def getThisElemForceMoment(pitch, radius, chord):
        elementParams = BladeElementParameters(shrimpParams.rho, pitch, width, radius,
                                               chord, bladeIndex, propParams.numBlades, 
                                               propParams.height_b2p, inflowVel, 
                                               prop_angVel)
        return getElementForceMoment(elementParams, state)
    
    forceMomentTuples_b = [getThisElemForceMoment(p, r, c) 
                           for (p, r, c) in zip(pitches, radii, chords)]
    forces_b = sum([f for f,_ in forceMomentTuples_b])
    moments_b = sum([m for _,m in forceMomentTuples_b])
    return (forces_b, moments_b)
    
def getPropForceMoment(propParams, shrimpParams, state):
    """Computes the force across all blades in the propellor
        Arguments:
        propParams: PropellorParameters to be iterated over
        shrimpParams: ShrimpParameters
        state: ShrimpState 
    """
    forceMomentTuples_b = [getBladeForceMoment(propParams, shrimpParams, state, i)
                           for i in range(propParams.numBlades)]
    forces_b = sum([f for f,_ in forceMomentTuples_b])
    moments_b = sum([m for _,m in forceMomentTuples_b])
    return (forces_b, moments_b)
    
def testPropForceMoment():
    numBlades = 2
    rho = 1.225
    pitchRootTip = (0, 0.15)
    radiusMax = 0.02
    radiusRootTip = (0.005, radiusMax)
    chordRootTip = (0.005, 0.004)
    height_b2p = 0.005
    propType = PropellorType.SHAFT
    propParams = PropellorParameters(numBlades, pitchRootTip, radiusRootTip, chordRootTip, 
                                     height_b2p, propType)
    
    mass = 0.005
    
    # Assuming disk radius
    Izz = 0.5*mass*radiusMax**2
    Ixx = Izz/2
    
    shrimpParams = ShrimpParameters(propParams, propParams, mass, Ixx, Izz, rho)
    
    state = dummyShrimpState()
    
    (f, m) = getPropForceMoment(propParams, shrimpParams, state)
    print('Forces: ', f)
    print('Moments: ', m)

testPropForceMoment()
    

TypeError: __init__() missing 4 required positional arguments: 'propIzz', 'rho', 'motorParams', and 'batteryParams'


## Multiple Inertial bodies
Of course, we don't have a single inertia, we have both the body and the shaft propellor. In this case, we calculate the Euler equations for each body separately. 
They are represented by a revolute joint, in which there is a motor torque and a counter motor torque

Reproducing the Euler equations:

$$\mathbf{I} {}^b \dot{\omega}^w = \mathbf{{}^b\tau_{aero}} - \mathbf{{}^b\omega^w \times I {}^b\omega^w}$$

where:
* ${}^b\omega^w$ is the angular velocity of the body relative to the world in the body frame.
* ${}^b v_b^w$ is the velocity of the body relative to the world frame in the body frame.
* $\mathbf{I}$ is the inertia tensor in the body frame. 


For the body's integration, from Piccoli's simulation, the Body Rotational accerlation in the flyer frame

$${}^F \dot{\omega}_b = ({}^F I_b)^{-1} (^{F} M_{dragplate} - {}^F M_{motor})^T  $$

And then the rotor/propellor rotational acceleration in the rotor/propellor frame:

$${}^P \dot{\omega}_r = ({}^P I_P)^{-1} (^{P} M_{propellor} + {}^P M_{motor})^T  $$

Piccoli then calculates both gyroscopic moments:

$$M_{gyro_b} = -{}^F \omega_b \times ({}^F I_b ({}^F \omega_b + \dot{\phi})$$
$$M_{gyro_P} = -{}^F \omega_b \times ({}^F I_P ({}^F \omega_b + R_p^F (\dot{\phi} + spinRate))$$

And finally:
$${}^F \dot{\omega}_b = ({}^F I_P + {}^F I_b + {}^F I_{total})^{-1} \left( {}^F M_{propellor} + {}^F M_{dragplate} - [1\ 1\ 0]^T {}^F M_{motor} + M_{gyro_p} + M_{gyro_b} - {}^FI_b {}^F \dot{\omega}_b - {}^FI_p {}^F \dot{\omega}_p \right)$$
* $I_{total} = mass_{stator}*(||r_b^s||_2^2 -(S_s_f')*S_s_f)+ mass_{rotor}*(S_r_f*(S_r_f')*eye(3)-(S_r_f')*S_r_f));$


In [ ]:

def flyerODE(t, odeState, shrimpParams):
    """ This assembles everything for the ode integration and returns dx/dt in the Flyer Frame
        Arguments:
            t: a time instant. Unused, since this is a time-invariant system.
            x: the array containing the state vector. 
            The ODE state vector is as follows:
            odeState = [x, y, z, vx, vy, vz, eulerAngles ZYX, 
                        angVel_f, bodyYawRate, propAngRate]
            odeState[0] = x body position in world frame
            odeState[1] = y body position in world frame
            odeState[2] = z body position in world frame
            odeState[3] = x body velocity in world frame
            odeState[4] = y body velocity  in world frame
            odeState[5] = z body velocity  in world frame
            odeState[6] = Z ZYX euler angle in world frame
            odeState[7] = Y ZYX euler angle in world frame
            odeState[8] = X ZYX euler angle in world frame
            odeState[9] = p is x roll rate in flyer frame
            odeState[10] = q is y pitch rate in flyer frame
            odeState[11] = r is z yaw rate in flyer frame
            odeState[12] = bodyYawRate is body yaw rate wrt flyer frame
            odeState[13] = propAngRate is the prop speed wrt body frame (stator frame)
            
    """
    # Set up useful quantities
    m = shrimpParams.mass
    inertia_b = shrimpParams.inertia_b
    propInertia_p = shrimpParams.propInertia_b
    vel_b2w_w = odeState[3:6]
    euler_b2w_w = odeState[6:9]
    angvel_f2w_f = odeState[9:12]
    bodyYawRate = odeState[12]
    propAngRate = odeState[13]
    
    state = ShrimpState.fromODE(odeState)
    
    angvel_b2w_f = angvel_f2w_f + np.array([0, 0, bodyYawRate])
    
    rot_b2w = state.rot_b2w
    rot_w2b = np.transpose(rot_b2w)
    rot_b2f = state.rot_b2f
    
    vel_b2w_f = rot_w2f.dot(vel_b2w_w)
    
    # Quantities in Flyer frame
    # For the purposes of calculating prop inertia the flyer frame, we assume that
    # the propellor is spinning fast enough to be essentially symmetric, so 
    # we can just use the inertia in the propellor frame essentially 
    propInertia_f = propInertia_p
    inertia_f = rotateTensor(inertia_b, rot_b2f)
    
    # Get velocities in body frame
    vel_b2w_b = rot_w2b.dot(vel_b2w_w)
    angvel_b2w_b = rot_w2b.dot(angvel_b2w_w)
    
    # Calculate Aerodynamic Forces
    bodyPropParams = shrimpParams.bodyPropParams
    shaftPropParams = shrimpParams.shaftPropParams
    (bodyForce_b, bodyMoment_b) = getPropForceMoment(bodyPropParams, shrimpParams, 
                                                     state)
    (shaftForce_b, shaftMoment_b) = getPropForceMoment(shaftPropParams, shrimpParams, 
                                                       state)
    bodyMoment_f = rot_b2f.dot(bodyMoment_b)
    shaftMoment_f = rot_b2f.dot(shaftMoment_b)
    forcesAero_b = bodyForce_b + shaftForce_b  # 1x3 numpy array
    momentsAero_b = bodyMoment_b + shaftMoment_b  # 1x3 numpy array
    
    # Calculate Gravity
    gravity_w = np.array([0, 0, -9.81])
    gravity_f = rot_w2f.dot(gravity_w)
    
    # Newton's equations
    acc_b2w_f = (1/m) * (forcesAero_f + m*gravity_f - m*np.cross(angvel_f2w_f, vel_f2w_f))
    
    ## Motor equations
    K_t = shrimpParams.motorParams.K_t
    motorR = shrimpParams.motorParams.resistance
    battR = shrimpParams.batteryParams.resistance
    battV = shrimpParams.batteryParams.voltage
    totalR = max(1e-6, pwm * pwm * battR + motorR) # Don't divide by zero
    motorCurrent = (battV * pwm - K_t * propAngRate) / totalR
    motorMoment_p = np.array([0 0 K_t * motorCurrent])
    # In piccoli code, Rr_f or rot_p2f is simply the rotation due to angle_r, 
    # or angle from rotor to stator frame.
    # This honestly doesn't matter since the motor moment doesn't change
    # so motorMoment_P = motorMoment_F, since z-axis is aligned!
    motorMoment_f = motorMoment_p
    
    ## Rotational Aspects
    # omg_rDot - prop z-rotational acc in prop frame
    angacc_prop_p = np.invert(propInertia_f).dot((rot_p2f.dot(shaftMoment_f) + 
                                               motorMoment_p).dot([0, 0, 1]))
    # omg_bDot - body z-rotational acc in flyer frame
    angacc_body_f = np.invert(inertia_b).dot((bodyMoment_f - motorMoment_f).dot([0, 0, 1]))
    # MGyro1
    bodyYawRate = odeState[12]
    moment_gyro1_f = -np.cross(angvel_b2w_f, inertia_f.dot(angvel_b2w_f + bodyYawRate))
    # MGyro2
    propAng_f = rot_p2f(np.array([0, 0, bodyYawRate + propAngRate]))
    moment_gyro2_f = -np.cross(angvel_b2w_f, propInertia_f.dot(propAng_f))
    allMoments_f = (bodyMoment_f + propMoment_f - motorMoment_f.dot([1 1 0]) \ 
                    + moment_gyro1_f + moment_gyro2_f - inertia_f.dot(angacc_body_f) \
                    - propInertia_f.dot(angacc_prop_p))
    
    allInertiaInv_f = np.invert(inertia_f + propInertia_f)
    # Picocli's Euler equations
    angacc_b2w_f = allInertiaInv_f.dot(allMoments_f)
    
    
    
    

In [ ]:
    ## Euler's equations - DEPRECATED to try out piccoli's euler
    # angacc_b2w_b = (np.inv(inertia_b) * \
    #              (momentsAero_b - np.cross(angvel_b2w, inertia.dot(angvel_b2w))))
    ## Transform force into the Euler frame
    # acc_b2w_w = rot_b2w.dot(acc_b2w_b)
    